In [1]:
import sys
sys.path.append("../src")

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

import sys
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.datasets import FashionMNIST

import matplotlib.pyplot as plt
from pytorch_impl.nns import FCN
from pytorch_impl.nns.utils import warm_up_batch_norm, to_one_hot
from pytorch_impl.estimators import MatrixExpEstimator
from pytorch_impl import ClassifierTraining
from pytorch_impl.matrix_exp import matrix_exp, compute_exp_term

In [4]:
torch.manual_seed(0)

num_classes = 10

if torch.cuda.is_available() and False:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('Torch version: {}'.format(torch.__version__))
print('Device: {}'.format(device))

train_loader = torch.utils.data.DataLoader(
    FashionMNIST(root='.', train=True, download=True,
          transform=transforms.ToTensor()),
    batch_size=32, shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    FashionMNIST(root='.', train=False, transform=transforms.ToTensor()),
    batch_size=32, shuffle=True, pin_memory=True)

device

Torch version: 1.3.1
Device: cpu


device(type='cpu')

In [5]:
a = 30 / 180 * np.pi
M = torch.tensor([[0, -1], [1, 0]]) * a
matrix_exp(M, device).cpu().numpy()

array([[ 0.8660254, -0.5      ],
       [ 0.5      ,  0.8660254]], dtype=float32)

In [6]:
a = 30 / 180 * np.pi
M = torch.tensor([[0, -1], [1, 0]]) * a
M_clone = M.clone().to(device)
(torch.matmul(M_clone, compute_exp_term(M, device)) + torch.eye(2)).numpy()

array([[ 0.8660254, -0.5      ],
       [ 0.5      ,  0.8660254]], dtype=float32)

In [9]:
D = 28

model     = FCN(10, D * D).to(device)
estimator = MatrixExpEstimator(model, num_classes, device, criterion=nn.CrossEntropyLoss(), learning_rate=1., momentum=0)

In [12]:
estimator.predict(X)
for _ in range(200):
    _, (X, y) = next(enumerate(train_loader))
    X, y = X.to(device), y.to(device)
    estimator.fit(X, y)
    print(estimator.predict(X)[:5])
    print()
    print()

accuracy 0.87500, loss 0.01531
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3414092063903809
tensor([[-0.3338, -0.9810, -0.4215, -0.8494, -1.7486,  4.1046, -0.7653,  0.7009,
          0.0750,  0.2386],
        [-0.6081, -1.1190, -0.6937, -1.4812, -1.3826,  1.5553, -2.0984,  1.6034,
         -0.3516,  4.3019],
        [-1.6277, -1.2654, -1.6431, -0.8292, -1.0373,  3.7274, -0.4105,  0.8392,
          0.9327,  1.0883],
        [-1.9510, -1.0068, -1.0052, -1.3552, -1.0315,  6.8800, -0.5135,  0.2510,
          0.0382, -0.5826],
        [-1.7724, -2.0050,  1.5949,  6.1843,  1.0475, -1.8399,  3.1398, -1.1612,
         -2.7629, -3.0375]])


accuracy 0.65625, loss 0.02426
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.4355376958847046
tensor([[ 1.1148e-01, -9.4621e-01,  2.4694e+00, -1.2983e+00,  5.3115e+00,
         -2.9944e+00,  2.2582e+00, -1.6517e+00, -2.2793e+00, -1.4289e+00],
        [-3.2427e+00, -2.0414e+00, -3.8157e-01, -3.0114e+00,  2.9032e+0

accuracy 0.81250, loss 0.01601
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3259093761444092
tensor([[ 6.0202, -0.3650,  0.3566, -1.1426, -0.2221, -2.2159,  3.6583, -2.7572,
         -1.4087, -1.9534],
        [-4.6209, -2.3446,  1.2582, -1.4804,  0.4755,  1.1239, -0.2761, -1.5718,
          7.7352, -0.8159],
        [-1.2160, -1.1057, -0.0245, -1.0229, -0.8348,  5.0918, -0.2469,  1.4078,
         -0.2288, -1.9137],
        [-1.2220, -2.7371,  1.6504,  4.5872, -0.2406, -0.6995,  2.1876, -1.8351,
         -0.3798, -1.8946],
        [ 8.8103, -0.7873,  1.4893, -1.6496, -1.3290, -2.0013, -0.3549, -1.3799,
         -1.4516, -1.6282]])


accuracy 0.90625, loss 0.00664
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.141294002532959
tensor([[ 6.3349, -0.5101,  1.5697,  0.7678, -3.9745, -1.0151,  2.6074, -0.5263,
         -2.8489, -2.6245],
        [-0.8000,  3.0881,  0.7949,  6.4339, -2.3529, -2.1241, -2.7704, -1.7675,
         -0.7037, -0.4355],
   

computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.2642661333084106
tensor([[-2.7143, -0.2942,  2.8643, -3.3567,  5.6348, -1.9827,  2.4255, -0.2178,
         -0.5452, -1.8741],
        [ 4.6339, -0.7667,  0.7323, -0.4025, -0.2646, -1.6744,  2.1713, -1.4743,
         -2.0642, -1.2368],
        [-3.3764, -2.3182, -2.0480, -1.6180, -1.3575,  3.5630, -2.8348,  7.5840,
          1.1475,  0.6015],
        [-1.0831, -1.8573,  2.2573,  0.5944,  0.3891, -1.1974,  5.5607, -1.8869,
         -1.7595, -1.4201],
        [-3.6593, -0.4895, -1.1622,  3.4921,  5.8677, -3.3567,  2.6522, -1.4000,
         -0.5547, -1.7305]])


accuracy 0.87500, loss 0.02609
computing grads ... 1s
exponentiating kernel matrix ... 1s
beta = 1.4906269311904907
tensor([[ 2.5881e-01, -1.7935e+00, -3.4170e+00, -3.0959e+00, -5.1752e-01,
          1.9352e-01, -3.9505e+00,  3.1225e+00,  1.8136e+00,  6.8566e+00],
        [ 1.4491e+00,  1.1705e-01,  1.7793e-02,  1.2421e+00,  1.9912e+00,
         -3.8082e+00,  5.025

beta = 1.3740425109863281
tensor([[13.3175, -2.9621, -0.9755,  1.3968, -4.4277, -4.0676,  1.0401, -2.0955,
         -0.2069, -1.4446],
        [-0.5732, -1.0874,  7.5285, -0.1191, -0.2987, -4.1854,  2.3362, -1.0363,
         -1.2130, -1.6818],
        [-2.3612, -1.7306, -2.1908, -2.1756, -1.2418, -1.0890, -0.7874,  3.8176,
          0.4777,  7.0381],
        [ 3.8634,  0.3303, -2.4503,  6.7897, -1.4503, -2.2223,  0.5027, -1.9699,
         -3.4709, -0.4248],
        [-2.6723,  1.3848,  2.6096, -2.9526,  6.0307, -1.6009,  2.5949, -1.0681,
         -2.6863, -2.1209]])


accuracy 0.84375, loss 0.01773
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.2783401012420654
tensor([[-2.0291, -3.9428,  2.3014, -2.9976,  2.7531,  1.5771, -1.2045, -0.8275,
          6.5882, -2.5085],
        [-2.6154, 11.6089, -1.0208,  3.3692, -2.9012, -2.7944, -2.5340, -0.9230,
         -1.7369, -0.8697],
        [-2.2887, -1.0208, -1.0228, -0.4439, -1.6855,  0.9985, -2.1290,  6.9032,
         -0

accuracy 0.75000, loss 0.03525
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.6662330627441406
tensor([[-3.8730, -3.1946, -0.3340, -1.2333,  1.1827,  2.5161, -4.9683, -1.1716,
          8.4053,  1.8463],
        [-1.3179,  0.4071, -0.4197,  9.6696,  1.3439, -5.6206,  2.1726, -3.5120,
         -1.8110, -1.4289],
        [-3.7486, -2.1037,  0.6133,  0.6017, -0.5139,  0.3233, -1.6907,  1.1440,
          7.1533, -1.9790],
        [-4.4328, -0.1030,  3.3119, -2.3899, -3.7097,  2.4727, -0.7908, -2.0757,
         -6.3021, 13.2872],
        [ 7.4011, -0.7081,  1.4144, -1.4426, -0.4157, -3.3974,  2.1756, -2.4785,
         -0.1379, -2.9754]])


accuracy 0.75000, loss 0.02343
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3839386701583862
tensor([[-2.0034, -1.0665, -0.0367, -0.9119, -2.2737,  1.7095, -1.0344,  5.0844,
          0.4884, -0.2053],
        [-1.1342,  0.0709,  0.5213,  1.6361, -0.7256, -1.0372,  5.1631, -2.5519,
          0.2203, -2.5644],
  

...

accuracy 0.90625, loss 0.01727
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3198630809783936
tensor([[-0.8619, -1.1009, -1.7355,  0.0725, -2.7079,  1.2371,  0.6546, -1.4695,
         -6.0261, 11.3391],
        [-5.1011,  1.2181,  2.3503, -4.2375, 12.7966, -1.1615,  0.9385, -6.6075,
          0.2714, -0.9606],
        [-7.2477, -1.0132,  4.8911, -5.7153, 11.6225, -2.7750,  7.5604, -1.6837,
         -0.9624, -4.7953],
        [ 0.2448,  2.2969,  0.3895, 12.4727, -5.0897, -1.9104, -6.8657, -0.4161,
         -0.1185, -1.4998],
        [ 6.9826, -1.0465,  0.8744,  2.1440, -6.1117, -1.4038,  3.7823, -0.9878,
         -1.4093, -3.0035]])


accuracy 0.84375, loss 0.01280
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.2362383604049683
tensor([[-0.7518, -0.8615, -1.9252, -1.4259, -3.5181,  6.1881, -0.5002,  3.5537,
         -1.5955,  0.5447],
        [-6.1562, -1.6197,  9.2959, -3.4443,  6.6954, -4.3447,  3.3554, -1.4437,
          0.3535, -3.2941],
  

beta = 1.4178884029388428
tensor([[ 5.0428e+00, -9.5008e-01, -3.0890e-03, -1.1825e+00, -7.7629e-01,
         -4.5464e-01,  1.1515e+00,  8.1002e-01, -8.3577e-01, -2.9454e+00],
        [ 1.7291e+00,  2.1355e+00, -3.4346e+00,  1.0020e+01, -1.7504e+00,
         -2.6427e+00, -1.9504e+00, -3.9764e+00,  7.4391e-01, -1.2372e+00],
        [-1.1958e+00, -1.2646e+00, -3.1707e+00, -2.1516e+00, -2.4462e+00,
          2.3943e+00, -6.1021e-01,  1.0829e+01, -1.6482e+00, -9.7069e-01],
        [-1.6776e+00, -1.3058e+00,  7.0607e+00, -4.1627e+00,  1.0402e+01,
         -4.6994e+00,  7.4073e-02, -3.7513e+00, -1.0559e+00, -1.4772e+00],
        [-2.8932e+00,  3.1268e-01, -5.3224e+00, -1.7055e+00, -6.8879e+00,
          7.4225e+00, -2.4134e+00,  6.3739e-01, -4.5024e+00,  1.5107e+01]])


accuracy 0.84375, loss 0.02596
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3898147344589233
tensor([[-2.3155, -3.6791,  2.4065, -3.8966, 11.1304, -2.8031,  6.9346, -5.2630,
         -0.6151, -2.2135],
 

accuracy 0.81250, loss 0.01762
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3116955757141113
tensor([[ 0.7227, -2.5265, -4.2465, -3.7941, -1.3045, -1.0135,  1.0504, -0.4586,
         11.2916,  0.0149],
        [-3.9742, -1.3801, -2.8002, -0.3006, -2.4518,  5.3976, -6.2190, 10.6447,
         -0.2281,  0.7958],
        [-5.5973, -2.1490, -0.8354,  0.0508, -1.9977,  5.0816,  1.2905,  1.0294,
          1.6015,  1.3295],
        [-1.9955, 10.9051, -1.6090,  0.0909, -0.9567, -2.4897, -1.2072,  0.2443,
         -1.5712, -1.7140],
        [ 2.4954, -0.5500, -2.1582,  7.0348, -3.4578,  0.9609, -3.7710, -2.1871,
          2.5198, -1.2543]])


accuracy 0.78125, loss 0.02429
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3250354528427124
tensor([[ 2.1117e+00,  3.9362e-01, -8.7497e-01,  2.0218e+00, -2.3313e+00,
         -1.2390e+00,  6.3573e+00, -1.6520e+00, -1.8151e+00, -3.2405e+00],
        [-3.8075e+00, -2.1410e+00, -3.1499e+00, -3.3887e+00, -1.7854e+0

accuracy 0.87500, loss 0.01365
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3047198057174683
tensor([[-1.3436, -1.5222, -1.6302, -1.4785, -1.6857,  3.7070, -0.7870,  8.3363,
         -0.5684, -3.1221],
        [-5.6477,  1.6282,  4.7852, -6.4598, 12.5162, -5.7624,  5.6083, -2.5135,
         -1.5906, -3.1170],
        [-1.7463, -1.3331, -1.0133, -0.1693, -2.5237,  6.7964,  1.4619, -0.5059,
         -0.6007, -0.4476],
        [11.3484, -1.2536, -2.9346,  3.6136, -1.7225, -7.5598,  4.6707, -2.0006,
         -2.6119, -1.8632],
        [-1.7395, -1.1915, -0.4385, -1.2680, -3.2525,  7.7824, -0.7687,  2.2158,
          0.4156, -1.9515]])


accuracy 0.93750, loss 0.00489
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.1025035381317139
tensor([[-1.4880, -2.1197,  5.5077, -0.5015, 11.5826, -5.1996,  1.1183, -6.8685,
         -0.8974, -1.7461],
        [-1.4457, -0.5113,  3.3439, -3.0866,  9.4441, -1.5323,  1.6268, -1.7146,
         -2.3323, -3.7459],
  

accuracy 0.78125, loss 0.02493
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3432649374008179
tensor([[ 4.8606,  8.2131,  0.1531,  3.7629, -2.8281, -3.4040, -2.5108, -2.9220,
         -3.1278, -2.4614],
        [-3.9778,  2.1741,  8.7028,  0.0231,  4.9452, -3.0777, -2.4172, -0.7847,
         -2.5939, -3.7799],
        [-0.8536, -1.7672, -0.6069, -0.0978,  4.6173, -2.5319,  7.7916, -4.0346,
         -1.3024, -1.5275],
        [ 2.6699,  6.5128, -2.6813, 10.5363, -4.7622, -3.2446, -4.5646, -2.7979,
         -0.7358, -1.5056],
        [-2.3900, -1.8950, -0.0622, -0.6555, -1.3960,  6.9098, -1.0250, -1.0190,
          1.7650, -0.5180]])


accuracy 0.84375, loss 0.01730
computing grads ... 0s
exponentiating kernel matrix ... 0s
beta = 1.3102726936340332
tensor([[-0.9202, -2.4238, -0.2205, -1.8054, -4.8752,  2.7172, -7.7443,  3.0605,
         -0.6301, 12.3208],
        [-0.5834, -2.1460, -0.4005,  0.3307, -2.7145,  5.2700, -3.6250,  2.2606,
          1.0659,  0.2785],
  